<a href="https://colab.research.google.com/github/pko89403/Recsys_test/blob/master/item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pwd

/content


In [0]:
!ls /content/

sample_data


In [0]:
import pandas as pd

RATINGS_PATH = "/content/drive/My Drive/data/ml-1m/ratings.dat"
MOVIES_PATH = "/content/drive/My Drive/data/ml-1m/movies.dat"

In [0]:
ratings = pd.read_csv(RATINGS_PATH, header=None, sep="::", engine="python")
movies = pd.read_csv(MOVIES_PATH, header=None, sep="::", engine="python")

In [0]:
ratings.columns = ["uuid", "iid", "rating", "timestamp"]
movies.columns = ["iid", "movie_name", "genre"]

In [30]:
ratings.head()
movies.head()

,iid,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
iid_to_movie_name = dict( zip(movies.iid.tolist(), movies.movie_name.tolist()))
movie_name_to_iid = { v: k for k, v in iid_to_movie_name.items()}

In [0]:
iid_to_movie_name

In [0]:
movie_name_to_iid

Total Items는 중요하다. Total Items가 Word2Vec에서의 Vocab Size와 같다.

In [53]:
only_user = ratings.uuid.unique().tolist()
total_items = len(movie_name_to_iid)
print(f"Total Users : {len(only_user)}, Total Items : {total_items}")


Total Users : 6040, Total Items : 3883


유저 하나당 평점 매긴 영화들의 리스트를 만든다.    
왜냐면 그걸로 Item2Vec의 데이터로 사용할 것이기 때문에

In [0]:
skip_item_dataset = [ratings[ratings.uuid==user]['iid'].tolist() for user in only_user]

In [0]:
with open('/content/drive/My Drive/data/ml-1m/skip_item.dat', 'w') as f:
  for line in skip_item_dataset:
    f.write(str(line)+'\n')

In [0]:
def proc_item_freq(skip_item_dataset):
  total_items_with_dup = 0
  item_frequency = dict()
  for rate_hist in skip_item_dataset:
    for item in rate_hist:
      total_items_with_dup += 1
      try:
        item_frequency[item] += 1
      except:
        item_frequency[item] = 1
  return item_frequency, total_items_with_dup

In [0]:
item_freq, total_items_with_dup = proc_item_freq(skip_item_dataset)

In [0]:
import numpy as np 

def create_sample_table(item_frequency):
  sample_table_size = 1e6

  freq_dist = [ f ** 0.75 for f in item_frequency.values()]
  prob_dist = np.array(freq_dist) / sum(freq_dist)
  count_dist = np.round( prob_dist * sample_table_size)

  sample_table = list()
  for iid, cnt in enumerate(count_dist):
    sample_table += ( [iid] * int(cnt) )
  
  return sample_table

In [0]:
sample_table = create_sample_table(item_freq) 

In [0]:
def get_positive_pair(dataset, window_size, batch_size):
  pairs = list()
  
  while(len(pairs) < batch_size):
    open_file = dataset
    rate_hist = list()
    
    for line in open_file:
      rate_hist = line 

    for pos, u in enumerate(rate_hist):
      for v in rate_hist[max(pos-window_size,0): min(pos+window_size,len(rate_hist))]:
        if u == v:
          continue
        pairs.append((u, v))
    
  
  positive_pairs = random.sample(pairs,batch_size)

  return positive_pairs

import random 
def negative_samples(sample_table, k=5):
  return random.sample(sample_table, k)




In [0]:
positive_pairs = get_positive_pair(dataset=skip_item_dataset, window_size=3, batch_size=1024)

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F 

In [0]:
class SGNS(nn.Module):
  def __init__(self, total_items, embedding_dims):
    super(SGNS, self).__init__()
    self.u_embedding = nn.Embedding(total_items, embedding_dims)
    self.v_embedding = nn.Embedding(total_items, embedding_dims)
  
  def forward(self, u_idx, v_pos_idx, v_neg_idx):
    u_emb_vec = self.u_embedding(Variable(torch.LongTensor(u_idx)))
    v_pos_emb_vec = self.v_embedding(Variable(torch.LongTensor(v_pos_idx)))

    pos_pred = torch.mul(u_emb_vec, v_emb_vec).squeeze()
    pos_pred = torch.sum(pred, dim=1)
    pos_pred = F.logsigmoid(pred)

    v_neg_emb_vec = self.v_embedding(Variable(torch.LongTensor(v_neg_idx)))
    neg_pred = torch.bmm(v_neg_emb_vec, u_emb_vec.unsqueeze(2)).squeeze()
    neg_pred = F.logsigmoid(-1 * neg_pred)

    return -1 * ( torch.sum(pred) + torch.sum(neg_pred))